# Classification Model - SOLUTION

**Instructor Solution Notebook**

This notebook contains the complete solution for the classification problem using the **Customer Churn Dataset**.

---

## Table of Contents
1. [Setup & Imports](#setup)
2. [Data Loading & Exploration](#data-loading)
3. [Data Engineering](#data-engineering)
4. [Model Training](#model-training)
5. [Model Evaluation](#model-evaluation)
6. [Model Saving](#model-saving)
7. [Conclusions](#conclusions)

---
## 1. Setup & Imports <a id='setup'></a>

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
from imblearn.over_sampling import SMOTE

# Helper modules
import sys
sys.path.append('../src')
from data_engineering import *
from model_utils import *

# Set random seed
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print("All libraries imported successfully!")

---
## 2. Data Loading & Exploration <a id='data-loading'></a>

### 2.1 Load Dataset

In [ ]:
# SOLUTION: Using Customer Churn Dataset
DATA_PATH = '../data/raw/customer_churn.csv'

# Load data
df = load_data(DATA_PATH)

# Display first few rows
df.head()

### 2.2 Basic Data Exploration

In [ ]:
# Dataset shape
print(f"Dataset Shape: {df.shape[0]} rows, {df.shape[1]} columns\n")

# Column information
print("Column Information:")
print(df.info())

# Statistical summary
print("\nStatistical Summary:")
df.describe()

### 2.3 Identify Target Variable

In [ ]:
# SOLUTION: Target column for Customer Churn dataset
TARGET_COLUMN = 'churn'

# Check target distribution
print(f"Target Variable: {TARGET_COLUMN}")
print(f"\nValue Counts:")
print(df[TARGET_COLUMN].value_counts())
print(f"\nPercentage Distribution:")
print(df[TARGET_COLUMN].value_counts(normalize=True) * 100)

# Visualize target distribution
plt.figure(figsize=(8, 5))
df[TARGET_COLUMN].value_counts().plot(kind='bar', color=['#2ecc71', '#e74c3c'])
plt.title(f'Distribution of {TARGET_COLUMN}')
plt.xlabel(TARGET_COLUMN)
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

# Check for class imbalance
imbalance_ratio = df[TARGET_COLUMN].value_counts().min() / df[TARGET_COLUMN].value_counts().max()
print(f"\nClass Imbalance Ratio: {imbalance_ratio:.3f}")
if imbalance_ratio < 0.5:
    print("⚠️ Dataset is imbalanced. We'll need to handle this!")

### 2.4 Check for Missing Values

In [ ]:
# Check missing values
missing_summary = check_missing_values(df)

### 2.5 Identify Feature Types

In [ ]:
# SOLUTION: Identify feature types
feature_types = get_feature_types(df)

numerical_features = feature_types['numerical']
categorical_features = feature_types['categorical']

---
## 3. Data Engineering <a id='data-engineering'></a>

### 3.1 Handle Missing Values

In [ ]:
# SOLUTION: Handle missing values if any
if len(missing_summary) > 0:
    df = handle_missing_values(df, strategy='median')
else:
    print("No missing values to handle!")

### 3.2 Drop Irrelevant Columns

In [ ]:
# SOLUTION: Drop customer_id as it's not useful for prediction
columns_to_drop = ['customer_id'] if 'customer_id' in df.columns else []

if columns_to_drop:
    df = df.drop(columns=columns_to_drop)
    print(f"Dropped columns: {columns_to_drop}")

print(f"\nRemaining columns: {df.shape[1]}")
print(df.columns.tolist())

### 3.3 Encode Categorical Variables

In [ ]:
# SOLUTION: Encode categorical variables
# Update categorical features list after dropping columns
categorical_features = [col for col in df.select_dtypes(include=['object']).columns 
                       if col != TARGET_COLUMN]

if categorical_features:
    print(f"Encoding categorical features: {categorical_features}")
    df = encode_categorical(df, categorical_features, method='onehot')
else:
    print("No categorical features to encode")

print(f"\nShape after encoding: {df.shape}")

### 3.4 Separate Features and Target

In [ ]:
# SOLUTION: Separate X and y
X = df.drop(columns=[TARGET_COLUMN])
y = df[TARGET_COLUMN]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns ({len(X.columns)}):")
print(X.columns.tolist())

### 3.5 Train-Test Split

In [ ]:
# SOLUTION: Create train-test split with stratification
X_train, X_test, y_train, y_test = create_train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

# Verify class distribution
print(f"\nTraining set class distribution:")
print(y_train.value_counts(normalize=True))
print(f"\nTest set class distribution:")
print(y_test.value_counts(normalize=True))

### 3.6 Feature Scaling

In [ ]:
# SOLUTION: Scale features using StandardScaler
X_train_scaled, X_test_scaled, scaler = scale_features(
    X_train, X_test, 
    method='standard'
)

# Convert back to DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("\nFeatures scaled successfully!")
print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")

### 3.7 Handle Class Imbalance

In [ ]:
# SOLUTION: Apply SMOTE if dataset is imbalanced
imbalance_ratio = y_train.value_counts().min() / y_train.value_counts().max()

if imbalance_ratio < 0.5:
    print("Applying SMOTE to handle class imbalance...")
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
    
    print(f"Original training set: {X_train_scaled.shape[0]} samples")
    print(f"Resampled training set: {X_train_resampled.shape[0]} samples")
    print(f"\nClass distribution after SMOTE:")
    print(pd.Series(y_train_resampled).value_counts())
    
    X_train_final = X_train_resampled
    y_train_final = y_train_resampled
else:
    print("Class balance is acceptable, no SMOTE needed")
    X_train_final = X_train_scaled
    y_train_final = y_train

---
## 4. Model Training <a id='model-training'></a>

### 4.1 Train Multiple Models

In [ ]:
# SOLUTION: Train three classification models
trained_models = train_classification_models(X_train_final, y_train_final)

print("\nAll models trained successfully!")
print(f"Models: {list(trained_models.keys())}")

### 4.2 Compare Models on Test Set

In [ ]:
# SOLUTION: Compare all models
comparison_df = compare_classification_models(trained_models, X_test_scaled, y_test)

# Display comparison
comparison_df

### 4.3 Select Best Model

In [ ]:
# SOLUTION: Select best model based on F1 score
best_model_name = comparison_df.loc[comparison_df['f1'].idxmax(), 'Model']
best_model = trained_models[best_model_name]

print(f"✅ Best Model: {best_model_name}")
print(f"\nBest Model Metrics:")
print(comparison_df[comparison_df['Model'] == best_model_name].to_string(index=False))

---
## 5. Model Evaluation <a id='model-evaluation'></a>

### 5.1 Detailed Evaluation

In [ ]:
# SOLUTION: Get predictions and detailed evaluation
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]

print(f"Classification Report - {best_model_name}")
print("="*60)
print(classification_report(y_test, y_pred))

### 5.2 Confusion Matrix

In [ ]:
# SOLUTION: Plot confusion matrix
plot_confusion_matrix(best_model, X_test_scaled, y_test, best_model_name)

### 5.3 ROC Curve

In [ ]:
# SOLUTION: Plot ROC curve
plot_roc_curve(best_model, X_test_scaled, y_test, best_model_name)

### 5.4 Feature Importance

In [ ]:
# SOLUTION: Get and plot feature importance
if hasattr(best_model, 'feature_importances_'):
    feature_importance_df = get_feature_importance(
        best_model, 
        X_train_final.columns if isinstance(X_train_final, pd.DataFrame) else X_train_scaled.columns, 
        top_n=10
    )
else:
    print(f"{best_model_name} does not have feature importance attribute")

---
## 6. Model Saving <a id='model-saving'></a>

### 6.1 Save the Best Model

In [ ]:
# SOLUTION: Save best model
model_filename = f"../models/classification_{best_model_name.lower().replace(' ', '_')}.pkl"
save_model(best_model, model_filename)

### 6.2 Save Preprocessing Artifacts

In [ ]:
# SOLUTION: Save scaler
import joblib
joblib.dump(scaler, '../models/scaler_classification.pkl')
print("✅ Scaler saved to ../models/scaler_classification.pkl")

### 6.3 Save Model Performance Summary

In [ ]:
# SOLUTION: Save comparison results
comparison_df.to_csv('../models/classification_model_comparison.csv', index=False)
print("✅ Model comparison saved to ../models/classification_model_comparison.csv")

---
## 7. Conclusions <a id='conclusions'></a>

### Summary of Results

## SOLUTION SUMMARY

### 1. Dataset
- Used **Customer Churn Dataset** with ~10,000 customers
- Target: Predict whether a customer will churn (leave the service)

### 2. Data Challenges
- **Class Imbalance**: Dataset was imbalanced (minority class < 30%)
- **Solution**: Applied SMOTE to balance training data
- **Categorical Features**: Encoded country and gender using one-hot encoding
- **Feature Scaling**: Applied StandardScaler for numerical features

### 3. Best Model
- **Winner**: Random Forest or XGBoost (typically performs best)
- **Why**: Better handles non-linear relationships and feature interactions
- Tree-based models naturally handle different feature scales

### 4. Key Metrics
Expected performance:
- **Accuracy**: 82-86%
- **Precision**: 75-80%
- **Recall**: 70-75%
- **F1 Score**: 72-77%
- **ROC-AUC**: 0.85-0.90

### 5. Important Features
Top predictors of churn (typical findings):
1. **Age** - Older customers more likely to churn
2. **Number of Products** - Customers with 1 product more likely to churn
3. **Balance** - Very high or very low balance indicates churn risk
4. **Active Member** - Inactive members much more likely to churn
5. **Geography** - Location impacts churn rates

### 6. Model Limitations
- Model may struggle with edge cases (very young/old customers)
- Cannot capture temporal patterns (no time-series features)
- May have false positives for loyal customers with unusual patterns
- Performance depends heavily on feature quality

### 7. Business Insights
- Focus retention efforts on inactive customers
- Customers with only 1 product are high risk
- Age-specific retention strategies may be beneficial
- Geography-based campaigns could improve retention

### 8. Next Steps
1. **Feature Engineering**: Create interaction features (age × products, balance × tenure)
2. **Hyperparameter Tuning**: Grid search for optimal parameters
3. **Ensemble Methods**: Combine multiple models
4. **Cost-Sensitive Learning**: Assign higher cost to false negatives
5. **Deployment**: Deploy to H2O for production use
6. **Monitoring**: Track model performance over time

### Test Model Loading

In [ ]:
# SOLUTION: Verify model can be loaded and used
loaded_model = load_model(model_filename)

# Test prediction
test_prediction = loaded_model.predict(X_test_scaled[:5])
test_proba = loaded_model.predict_proba(X_test_scaled[:5])[:, 1]

print(f"\nTest predictions: {test_prediction}")
print(f"Prediction probabilities: {test_proba}")
print(f"Actual values: {y_test[:5].values}")
print("\n✅ Model loaded and tested successfully!")

---
## 🎉 Solution Complete!

This solution demonstrates:
- ✅ Complete data preprocessing workflow
- ✅ Handling class imbalance with SMOTE
- ✅ Training and comparing multiple models
- ✅ Comprehensive model evaluation
- ✅ Proper model packaging and saving
- ✅ Business insights and recommendations

**Expected Student Outcomes:**
- Similar results with their chosen dataset
- Understanding of complete ML workflow
- Ability to interpret and explain results
- Ready for deployment phase